In [20]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
#from wordcloud import WordCloud 
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns

import numpy as np 
import pandas as pd

import random as rn
from transformers import TFDistilBertModel
from transformers import DistilBertTokenizer

In [2]:
#tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
#data = pd.read_csv('./show.csv')
data = pd.read_parquet(r'C:\Shared\Final\Spark\Sleep',engine="pyarrow")
# data.head()

In [6]:
load_model = tf.keras.models.load_model(r"C:\Users\Student\Desktop\tableau_dataset\result\模型\DistrilBert_Sleep.h5")

In [7]:
data.head()

,Hotel_Name,Average_Rating,Review_Date,Stay_Date,Review_Title,Total_Rating,Review_Text,Reviewer_Name,Sleep_Quality,Hotel_City,Country
0,Delfino Blu Wellness Boutique Hotel,5.0,2021-08-31,2021-08-31,Felt truly looked after in welcoming boutique ...,5,The wonderful staff team quickly get to know a...,Liverpool Logs,5,Dhermi,Albania
1,Delfino Blu Wellness Boutique Hotel,5.0,2020-10-04,2020-09-30,Paradise in Europe,5,We have just returned from our first visit to ...,Sue & John,5,Dhermi,Albania
2,Delfino Blu Wellness Boutique Hotel,5.0,2019-08-30,2019-08-31,Family Wilson Delfino Blu 2019,5,"As we go here every year, if possible, I think...",Andy W,5,Dhermi,Albania
3,Delfino Blu Wellness Boutique Hotel,5.0,2019-08-27,2019-07-31,Beautiful,5,Second visit to this amazing hotel. From the ...,Irene T,5,Dhermi,Albania
4,Delfino Blu Wellness Boutique Hotel,5.0,2019-08-16,2019-08-31,"Great Hotel, staff and food",5,"We felt welcomed, looked after, and spoilt fro...",BertieEastAnglia,4,Dhermi,Albania


In [28]:
x=len(data)/10

In [32]:
data.head()

,Hotel_Name,Average_Rating,Review_Date,Stay_Date,Review_Title,Total_Rating,Review_Text,Reviewer_Name,Sleep_Quality,Hotel_City,Country
0,Delfino Blu Wellness Boutique Hotel,5.0,2021-08-31,2021-08-31,Felt truly looked after in welcoming boutique ...,5,The wonderful staff team quickly get to know a...,Liverpool Logs,5,Dhermi,Albania
1,Delfino Blu Wellness Boutique Hotel,5.0,2020-10-04,2020-09-30,Paradise in Europe,5,We have just returned from our first visit to ...,Sue & John,5,Dhermi,Albania
2,Delfino Blu Wellness Boutique Hotel,5.0,2019-08-30,2019-08-31,Family Wilson Delfino Blu 2019,5,"As we go here every year, if possible, I think...",Andy W,5,Dhermi,Albania
3,Delfino Blu Wellness Boutique Hotel,5.0,2019-08-27,2019-07-31,Beautiful,5,Second visit to this amazing hotel. From the ...,Irene T,5,Dhermi,Albania
4,Delfino Blu Wellness Boutique Hotel,5.0,2019-08-16,2019-08-31,"Great Hotel, staff and food",5,"We felt welcomed, looked after, and spoilt fro...",BertieEastAnglia,4,Dhermi,Albania


In [31]:
print(area)

[418555, 837110, 1255665, 1674220, 2092774, 2511329, 2929884, 3348439, 3766994, 4185549]


In [30]:
area = []
for i in range(10):
    area.append(round((i+1)*x))

In [ ]:
data.loc[0,41855]

In [ ]:
for i,a in enumerate(area):
    if i == 0:
        data1 = data.loc[0:a]
    else:
        data1 = data.loc[area[i-1]:a]
    X = data1['Review_Text'].copy().values.tolist()
    print(len(X))
    #tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    seq_len = 350
    
    # train_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)
    test_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)

    # input_ids_train = np.array(train_encodings['input_ids']) 
    # attention_mask_train = np.array(train_encodings['attention_mask'])

    input_ids_test = np.array(test_encodings['input_ids']) 
    attention_mask_test = np.array(test_encodings['attention_mask'])
    bert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    # Creating bert model
    inp_ids = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
    attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
    last_hidden_state = bert(inp_ids,attention_mask=attention_mask)[0] # Shape:(batch_size, seq_len, 768)
    out = last_hidden_state[:,0,:] # Shape:(Batch_size, 768)

    bert_model = tf.keras.Model(inputs=[inp_ids, attention_mask], outputs=out)

    # Passing data from bert pretrained model and extracting the final state.

    # print("Passing train data")
    # bert_output_train = bert_model.predict(
    #     [input_ids_train,attention_mask_train], batch_size=8, verbose=1)

    print("Passing test data")
    bert_output_test = bert_model.predict(
        [input_ids_test,attention_mask_test], batch_size=8, verbose=1)

    pred = load_model.predict(bert_output_test)
    pred_final = np.argmax(pred, axis=-1) + 1
    data["Service"] = pred_final
    data.to_csv("show"+i+".csv", index=False)


418556


In [35]:
X[0]

'a good clean hotel as you would expect .the first room we were offered was very dark facing a wall which was only feet away .i went to reception and was offered a swap which we were very pleased about .if there hadnt been another room avaliable i could not have stayed. all the staff were very professional and seemed happy in their work all young with a spring in their step.a special thanks to angelica who changed our room'

In [ ]:
    from transformers import DistilBertTokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    seq_len = 350
    
    # train_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)
    test_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)

    # input_ids_train = np.array(train_encodings['input_ids']) 
    # attention_mask_train = np.array(train_encodings['attention_mask'])

    input_ids_test = np.array(test_encodings['input_ids']) 
    attention_mask_test = np.array(test_encodings['attention_mask'])
    from transformers import TFDistilBertModel
    bert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    # Creating bert model
    inp_ids = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
    attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
    last_hidden_state = bert(inp_ids,attention_mask=attention_mask)[0] # Shape:(batch_size, seq_len, 768)
    out = last_hidden_state[:,0,:] # Shape:(Batch_size, 768)

    bert_model = tf.keras.Model(inputs=[inp_ids, attention_mask], outputs=out)

    # Passing data from bert pretrained model and extracting the final state.

    # print("Passing train data")
    # bert_output_train = bert_model.predict(
    #     [input_ids_train,attention_mask_train], batch_size=8, verbose=1)

    print("Passing test data")
    bert_output_test = bert_model.predict(
        [input_ids_test,attention_mask_test], batch_size=8, verbose=1)

    pred = load_model.predict(bert_output_test)
    pred_final = np.argmax(pred, axis=-1) + 1
    data["Service"] = pred_final
    data.to_csv("show"+i+".csv", index=False)

In [22]:
len(data1)

418555

In [82]:
data

,Hotel_Name,Average_Rating,Review_Date,Stay_Date,Review_Title,Total_Rating,Review_Text,Reviewer_Name,Hotel_City,Country,preds,Cleanliness,Location,Room,Sleep
0,Delfino Blu Wellness Boutique Hotel,5.0,2021-10-03,2021-09-30,A small slice of paradise,5,We stayed for 7 nights at the end of September...,Floyd @ Cambridge UK,Dhermi,Albania,5,5,5,5,5
1,Delfino Blu Wellness Boutique Hotel,5.0,2021-09-29,2021-09-30,A truly wonderful experience,5,We were looking for a relaxing week in the sun...,Liz A,Dhermi,Albania,5,5,5,5,5
2,Delfino Blu Wellness Boutique Hotel,5.0,2021-09-12,2021-08-31,Great Boutique Experience,5,We had been considering a trip here for some t...,Herts_foodie2,Dhermi,Albania,4,5,4,4,4
3,Delfino Blu Wellness Boutique Hotel,5.0,2021-08-31,2021-08-31,Felt truly looked after in welcoming boutique ...,5,The wonderful staff team quickly get to know a...,Liverpool Logs,Dhermi,Albania,5,5,5,5,5
4,Delfino Blu Wellness Boutique Hotel,5.0,2021-08-23,2021-08-31,Ultimate relaxation at beautiful Delfino Blu,5,"We just LOVED Delfino Blu, a little bit of Cor...",Peter Cummins,Dhermi,Albania,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Saranda Palace Hotel,4.5,2019-09-24,2019-09-30,Great place to stay in Saranda,5,We stayed here for 1 week. We had a triple roo...,XanderNieuw,Saranda,Albania,4,4,4,3,4
9996,Saranda Palace Hotel,4.5,2019-06-06,2019-06-30,Mediocre,3,"The pros: nice beach and pool, very clean.\nNo...",Trullala,Saranda,Albania,3,3,2,2,2
9997,Saranda Palace Hotel,4.5,2019-06-03,2019-05-31,Perfect vacation,5,Perfect place for a holiday if you don't want ...,Matkailija,Saranda,Albania,4,5,4,4,5
9998,Saranda Palace Hotel,4.5,2019-06-01,2019-05-31,Friendly staff and nice private beach area,3,The hotel is located in the outskirts of Saran...,Tina,Saranda,Albania,3,3,4,3,2


In [83]:
X = data1['Review_Text'].copy().values.tolist()
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

seq_len = 350

# train_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)
test_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)

# input_ids_train = np.array(train_encodings['input_ids']) 
# attention_mask_train = np.array(train_encodings['attention_mask'])

input_ids_test = np.array(test_encodings['input_ids']) 
attention_mask_test = np.array(test_encodings['attention_mask'])
from transformers import TFDistilBertModel
bert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
# Creating bert model
inp_ids = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
last_hidden_state = bert(inp_ids,attention_mask=attention_mask)[0] # Shape:(batch_size, seq_len, 768)
out = last_hidden_state[:,0,:] # Shape:(Batch_size, 768)

bert_model = tf.keras.Model(inputs=[inp_ids, attention_mask], outputs=out)

# Passing data from bert pretrained model and extracting the final state.

# print("Passing train data")
# bert_output_train = bert_model.predict(
#     [input_ids_train,attention_mask_train], batch_size=8, verbose=1)

print("Passing test data")
bert_output_test = bert_model.predict(
    [input_ids_test,attention_mask_test], batch_size=8, verbose=1)

pred = load_model.predict(bert_output_test)
pred_final = np.argmax(pred, axis=-1) + 1
data["Service"] = pred_final

In [84]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

seq_len = 350

# train_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)
test_encodings = tokenizer(X, truncation=True, padding=True, max_length=seq_len)

# input_ids_train = np.array(train_encodings['input_ids']) 
# attention_mask_train = np.array(train_encodings['attention_mask'])

input_ids_test = np.array(test_encodings['input_ids']) 
attention_mask_test = np.array(test_encodings['attention_mask'])

In [85]:
from transformers import TFDistilBertModel

bert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [86]:
# Creating bert model
inp_ids = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
last_hidden_state = bert(inp_ids,attention_mask=attention_mask)[0] # Shape:(batch_size, seq_len, 768)
out = last_hidden_state[:,0,:] # Shape:(Batch_size, 768)

bert_model = tf.keras.Model(inputs=[inp_ids, attention_mask], outputs=out)

# Passing data from bert pretrained model and extracting the final state.

# print("Passing train data")
# bert_output_train = bert_model.predict(
#     [input_ids_train,attention_mask_train], batch_size=8, verbose=1)

print("Passing test data")
bert_output_test = bert_model.predict(
    [input_ids_test,attention_mask_test], batch_size=8, verbose=1)

Passing test data
1250/1250 [==============================] - 52s 41ms/step


In [87]:
bert_output_test.shape

(10000, 768)

In [88]:
load_model = tf.keras.models.load_model("./模型/DistrilBert_filter_downsample_Service.h5")

In [89]:
pred = load_model.predict(bert_output_test)
pred_final = np.argmax(pred, axis=-1) + 1

In [90]:
pred_final

array([5, 5, 4, ..., 5, 4, 3], dtype=int64)

In [91]:
data["Service"] = pred_final

In [92]:
data.to_csv("show.csv", index=False)

In [93]:
pd.read_csv("show.csv")

,Hotel_Name,Average_Rating,Review_Date,Stay_Date,Review_Title,Total_Rating,Review_Text,Reviewer_Name,Hotel_City,Country,preds,Cleanliness,Location,Room,Sleep,Service
0,Delfino Blu Wellness Boutique Hotel,5.0,2021-10-03,2021-09-30,A small slice of paradise,5,We stayed for 7 nights at the end of September...,Floyd @ Cambridge UK,Dhermi,Albania,5,5,5,5,5,5
1,Delfino Blu Wellness Boutique Hotel,5.0,2021-09-29,2021-09-30,A truly wonderful experience,5,We were looking for a relaxing week in the sun...,Liz A,Dhermi,Albania,5,5,5,5,5,5
2,Delfino Blu Wellness Boutique Hotel,5.0,2021-09-12,2021-08-31,Great Boutique Experience,5,We had been considering a trip here for some t...,Herts_foodie2,Dhermi,Albania,4,5,4,4,4,4
3,Delfino Blu Wellness Boutique Hotel,5.0,2021-08-31,2021-08-31,Felt truly looked after in welcoming boutique ...,5,The wonderful staff team quickly get to know a...,Liverpool Logs,Dhermi,Albania,5,5,5,5,5,5
4,Delfino Blu Wellness Boutique Hotel,5.0,2021-08-23,2021-08-31,Ultimate relaxation at beautiful Delfino Blu,5,"We just LOVED Delfino Blu, a little bit of Cor...",Peter Cummins,Dhermi,Albania,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Saranda Palace Hotel,4.5,2019-09-24,2019-09-30,Great place to stay in Saranda,5,We stayed here for 1 week. We had a triple roo...,XanderNieuw,Saranda,Albania,4,4,4,3,4,4
9996,Saranda Palace Hotel,4.5,2019-06-06,2019-06-30,Mediocre,3,"The pros: nice beach and pool, very clean.\nNo...",Trullala,Saranda,Albania,3,3,2,2,2,3
9997,Saranda Palace Hotel,4.5,2019-06-03,2019-05-31,Perfect vacation,5,Perfect place for a holiday if you don't want ...,Matkailija,Saranda,Albania,4,5,4,4,5,5
9998,Saranda Palace Hotel,4.5,2019-06-01,2019-05-31,Friendly staff and nice private beach area,3,The hotel is located in the outskirts of Saran...,Tina,Saranda,Albania,3,3,4,3,2,4
